In [2]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import os
import numpy as np
import pandas as pd
import base64
from pymongo import MongoClient
from bson.json_util import dumps


# file name and class name import statement
from shelterClass import AnimalShelter






###########################
# Data Manipulation / Model
###########################
# Username and password to be used in class instance
username = "aacuser"
password = "user123"
shelter = AnimalShelter(username, password)


# class read method to return of cursor object 
df = pd.DataFrame.from_records(shelter.readDataBase({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')
student_id = "Corey Nance"

#Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# App layout to Place the HTML image tag according to design
# and a unique identifier of my name stored within student_id variable.

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Center(html.B(html.H3('Student_ID - ' + student_id))), # Unique identifier to be made visable
    html.Center(
        html.A(
            href='https://www.snhu.edu',
            children=[
                html.Img(
                    alt="Grazioso Salvare logo",
                    src='data:image/png;base64,{}'.format(encoded_image.decode()), # Grazioso Salvare Logo Link
                )
            ]
        )
    ),
            
#Code for the interactive filtering options, Radio buttons.
    html.Div(
        
        className='row',
             style={'display' : 'flex'},
             children=[
                 dcc.RadioItems(
                     id='filter-type',
                     options=[
                         {'label': 'Water Rescue', 'value': 'WR'},
                         {'label': 'Mountain Rescue', 'value': 'MR'},
                         {'label': 'Disaster Rescue', 'value': 'DR'},
                         {'label': 'Reset', 'value': 'RT'},
                ])
             ]),
    
   
     html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        
#Features for the interactive data table to make it user-friendly 
        editable=False,
        #filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
  
     html.Br(),
     html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
    
])

#############################################
# Interaction Between Components / Controller
#############################################


#Code to filter interactive data table with MongoDB queries
    
@app.callback(
    [Output('datatable-id','data'),
     Output('datatable-id','columns')],
[Input('filter-type', 'value')]
)

def update_dashboard(filter_type):
    
   
    if filter_type == 'WR':
           df = pd.DataFrame(list(shelter.readDataBase({"animal_type":"Dog","breed":
                                                      {"$in":["Labrador Retriever Mix",
                                                              "Chesapeake Bay Retriever",
                                                              "Newfoundland"]},
                                                      "sex_upon_outcome":"Intact Female",
                                                       "age_upon_outcome_in_weeks":{"$gte":26},
                                                        "age_upon_outcome_in_weeks":{"$lte":156}})
                                 ))
    elif filter_type == 'MR':
          df = pd.DataFrame(list(shelter.readDataBase({"animal_type":"Dog","breed":
                                                      {"$in":["German Shepherd",
                                                              "Malamute",
                                                              "Old English Sheepdog",
                                                             "Siberian Husky",
                                                             "Rottweiler"]},
                                                      "sex_upon_outcome":"Intact Male",
                                                       "age_upon_outcome_in_weeks":{"$gte":26},
                                                        "age_upon_outcome_in_weeks":{"$lte":156}})
                                 ))
    elif filter_type == 'DR':
         df = pd.DataFrame(list(shelter.readDataBase({"animal_type":"Dog","breed":
                                                      {"$in":["Doberman Pinscher",
                                                              "German Shepherd",
                                                              "Golden Retriever",
                                                             "Bloodhound",
                                                             "Rottweiler"]},
                                                      "sex_upon_outcome":"Intact Male",
                                                       "age_upon_outcome_in_weeks":{"$gte":20},
                                                        "age_upon_outcome_in_weeks":{"$lte":300}})
                                 ))
    elif filter_type == 'RT':
         df = pd.DataFrame.from_records(shelter.readDataBase({}))
    else:
        df = pd.DataFrame.from_records(shelter.readDataBase({}))
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
    
    
    return (data,columns)
    

@app.callback(
     Output('graph-id', "children"),
     [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    
    # convert data to pandas dataframe
    dff = pd.DataFrame.from_dict(viewData)
    # count by breed occurrence
    new_df = dff.breed.value_counts()
    new_df.name = "breed"
    new_df = new_df.to_frame()
    new_df.reset_index(inplace=True)
    
    # return updated graph data
    
    return [
        dcc.Graph(
            figure = px.pie(new_df, values="breed", names="index")
        )
    ]

@app.callback(
     Output('map-id', "children"),
     [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#Code for your geolocation chart

    dff = pd.DataFrame.from_dict(viewData)
     # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[1,9]),# Settings changed so that the first dog in the list is displayed on map
                dl.Popup([
                    html.H1('Animal Name'),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]


app